In [1]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml

import pandas as pd 

import sqlite3

import pyperclip

import bs4
from bs4 import BeautifulSoup
import requests

import dotenv

import IPython
from IPython.display import HTML, Markdown, display

from atproto import Client

import PIL
from PIL import Image

# disable 'show more outputs' nonsense
from IPython.core.interactiveshell import InteractiveShell

# Always display all outputs in a cell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")


requests        2.32.3
BeautifulSoup   4.13.3


In [3]:
dotenv.load_dotenv()


True

# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)


In [4]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=80,
)


ProfileViewDetailed(did='did:plc:qomkdnxrqw3gkbytdxea5z65', handle='skynetandchill.com', associated=ProfileAssociated(chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreihi55ddvmohg6dztiwlkwdbts5fd76iaenef2sriacvj7ibtezagi@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreigu723bmkba3mff4krhcrrf7b5zw2svixwbvtj54v5ho64o2xpmzu@jpeg', created_at='2024-02-12T00:15:55.099Z', description='Daily artificial intelligence news, lovingly curated by man and machine. https://www.skynetandchill.com\n\nNeo-Luddite AI maven. On a long enough timeline, p(doom) for everything goes to 1.', display_name='SkynetAndChill.com', followers_count=335, follows_count=347, indexed_at='2024-11-21T15:40:08.470Z', joined_via_starter_p

In [5]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text


In [6]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        log("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        log("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        log("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        log("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        log("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        log("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        log("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        log("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        log("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        log("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        log("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        log("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        log("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response



In [7]:
def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)


In [8]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find trailing occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))


Elon Musk says we'll run out of power capacity to run all the AI chips in 2025


In [9]:
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        log(f"Error fetching {url}: {e}")
    return retdict


url = "https://druce.ai"
get_og_tags(url)

{'og:site_name': 'Druce.ai',
 'og:title': 'Druce.ai',
 'og:type': 'website',
 'og:description': "Druce's Blog on Machine Learning, Tech, Markets and Economics",
 'og:url': 'https://druce.ai/',
 'title': 'Druce.ai'}

In [10]:
def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            log(f'Failed to delete {file_path}. Reason: {e}')
            

In [11]:
# import sqlite3
# import tldextract

# # Path to your SQLite database
# db_path = "articles.db"

# # Connect to the database
# conn = sqlite3.connect(db_path)
# cur = conn.cursor()

# # Fetch all rows (id, hostname)
# cur.execute("SELECT id, hostname FROM sites")
# rows = cur.fetchall()

# for row in rows:
#     site_id, hostname = row
#     # Extract the registered domain (e.g., 'bbc.co.uk', '9to5mac.com')
#     extracted = tldextract.extract(hostname)
#     if extracted.domain and extracted.suffix:
#         registered_domain = f"{extracted.domain}.{extracted.suffix}"
#         # Update the domain column
#         print(f"{hostname} -> {registered_domain}")
#         cur.execute(
#             "UPDATE sites SET domain = ? WHERE id = ?",
#             (registered_domain, site_id))
#     else:
#         registered_domain = None  # fallback if parsing fails

# # Commit changes and close
# conn.commit()
# conn.close()

In [12]:
imgdir = 'download_images'  # for images
delete_files(imgdir)

conn = sqlite3.connect('articles.db')
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname: row.site_name for row in sites_df.itertuples()}
conn.close()


In [13]:
def parse_feed():
    display_dict = {}
    image_dict = {}
    for i, post in enumerate(data.feed):
        
        # check date
        date_str = data.feed[0].post.record.created_at
        dt = datetime.fromisoformat(date_str.rstrip('Z'))
        # break if more than 30 hours ago
        if dt  < datetime.now() - timedelta(hours=30):
            break

        # get post_str, post_url
        post_str = post.post.record.text.rstrip()
        post_str = truncate_last_occurrence(post_str)
        post_str = post_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape

        try:
            post_url = post.post.record.embed.external.uri.rstrip()
        except:
            post_url = ""

        if post_url:
            print(post_url) # to know if it's blocked
            # get site name , image using og tags
            try:
                tag_dict = get_og_tags(post_url)
            except:
                tag_dict = {}

            site_name = tag_dict.get('og:site_name')
            img_url = tag_dict.get('og:image')

            # get site_name from stored values as backup
            if not(site_name):
                parsed_url = urlparse(post_url)
                hostname = parsed_url.hostname
                site_name = sites_dict.get(hostname, hostname)

            md_str = f"[{post_str}]({post_url})  - {site_name}"

            display_dict[i]=md_str

            # write local copy of image
            if img_url:
                try:
                    r = rawfetchurl(img_url)
                    content_type = r.headers['Content-Type']
                    content_type = content_type[content_type.find('/')+1:]
                    impath = f"{imgdir}/source{i}.{content_type}"
                    with open(impath, 'wb') as file:
                        file.write(r.content)
                    output_path = f'{imgdir}/Image{i}.jpg'
                    resize_and_crop(impath, output_path)
                    image_dict[i] = f"![image]({output_path})\n\n"
                except Exception as e:
                    print(e)

        else: # no url
            md_str = f"\n{post_str}" + "\n\n"
            display_dict[i]=md_str
            
    return display_dict, image_dict

display_dict, image_dict = parse_feed()
len(display_dict)


https://www.darkdaily.com/2025/05/30/new-prostate-cancer-test-uses-machine-learning-to-efficiently-spot-new-cancer-biomarkers-accurately-and-non-invasively/
https://www.zdnet.com/article/ai-agents-make-great-teammates-but-dont-let-them-code-alone-heres-why/
https://www.technologyreview.com/2025/05/30/1117551/this-benchmark-used-reddits-aita-to-test-how-much-ai-models-suck-up-to-us/
https://www.hollywoodreporter.com/business/business-news/ai-napster-moment-1236232248/
https://bgr.com/business/nvidia-ceo-knows-exactly-who-will-take-your-job-and-its-not-just-ai/
https://www.ft.com/content/2fff10ea-f87e-4aaa-ae0e-50d17022dfa3
https://www.pymnts.com/artificial-intelligence-2/2025/openai-aims-to-offer-ai-access-through-ambient-computer-layer-without-screens/
https://www.cnet.com/tech/services-and-software/use-ai-at-work-your-coworkers-may-be-judging-you/
https://www.nytimes.com/2025/05/30/technology/ai-jobs-college-graduates.html
https://www.techradar.com/computing/i-tried-firefox-139s-new-a

80

In [14]:
output_str = ""
for i in sorted(display_dict.keys()):
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))


0. [Urine-based ML test identified prostate cancer biomarkers more accurately than PSA.](https://www.darkdaily.com/2025/05/30/new-prostate-cancer-test-uses-machine-learning-to-efficiently-spot-new-cancer-biomarkers-accurately-and-non-invasively/)  - www.darkdaily.com

---

1. [Carnegie-Mellon study reveals autonomous coding agents succeed on only 30% of tasks, reinforcing need for human collaboration.](https://www.zdnet.com/article/ai-agents-make-great-teammates-but-dont-let-them-code-alone-heres-why/)  - ZDNET![image](download_images/Image1.jpg)



---

2. [Elephant benchmark used Reddit’s AITA to test how much AI models suck up to us](https://www.technologyreview.com/2025/05/30/1117551/this-benchmark-used-reddits-aita-to-test-how-much-ai-models-suck-up-to-us/)  - MIT Technology Review![image](download_images/Image2.jpg)



---

3. [Is media coming for AI with a "Napster moment?"](https://www.hollywoodreporter.com/business/business-news/ai-napster-moment-1236232248/)  - The Hollywood Reporter![image](download_images/Image3.jpg)



---

4. [Huang says AI won't take your job, but someone who uses AI effectively might. Other CEOs chime in. Some categories of jobs have disappeared due to tech: phone operator, typist etc.](https://bgr.com/business/nvidia-ceo-knows-exactly-who-will-take-your-job-and-its-not-just-ai/)  - BGR![image](download_images/Image4.jpg)



---

5. [Could AI make a Scorsese movie? Demis Hassabis and Darren Aronofsky discuss](https://www.ft.com/content/2fff10ea-f87e-4aaa-ae0e-50d17022dfa3)  - Financial Times![image](download_images/Image5.jpg)



---

6. [OpenAI Aims to Offer AI Access Through ‘Ambient Computer Layer,’ Without Screens](https://www.pymnts.com/artificial-intelligence-2/2025/openai-aims-to-offer-ai-access-through-ambient-computer-layer-without-screens/)  - PYMNTS.com![image](download_images/Image6.jpg)



---

7. [People who use AI at work are silently judged more lazy, less competent](https://www.cnet.com/tech/services-and-software/use-ai-at-work-your-coworkers-may-be-judging-you/)  - CNET![image](download_images/Image7.jpg)



---

8. [For Some Recent Graduates, the A.I. Job Apocalypse May Already Be Here](https://www.nytimes.com/2025/05/30/technology/ai-jobs-college-graduates.html)  - The New York Times

---

9. [Firefox will summarize links if you hover and press alt-shift, but it's slow.](https://www.techradar.com/computing/i-tried-firefox-139s-new-ai-generated-link-previews-and-while-i-love-the-idea-im-skipping-them-for-one-key-reason)  - TechRadar![image](download_images/Image9.jpg)



---

10. [Business Insider fires 21% of staff in AI pivot "away from journalism toward greed"](https://www.avclub.com/business-insider-laysoff-20-percent-staff-ai)  - The A.V. Club

---

11. [AI generates navigable 3d environments like game levels.](https://www.zdnet.com/article/this-interactive-ai-video-generator-feels-like-walking-into-a-video-game-how-to-try-it/)  - ZDNET![image](download_images/Image11.jpg)



---

12. [TikTok boosts Polish hard right content ahead of elections](https://globalwitness.org/en/campaigns/digital-threats/tiktok-algorithm-recommends-twice-as-much-hard-right-content-to-users-ahead-of-polish-election/)  - Global Witness![image](download_images/Image12.jpg)



---

13. [CEOs know AI will shrink their teams in some cases 30-40% — they're just too afraid to say it, say 2 software investors](https://finance.yahoo.com/news/ceos-know-ai-shrink-teams-052955673.html)  - Yahoo Finance

---

14. 
AI will replace us all. Also AI:



---

15. [Google fixes bug that led AI Overviews to say it's now 2024](https://techcrunch.com/2025/05/30/google-fixes-bug-that-led-ai-overviews-to-say-its-now-2024/)  - TechCrunch![image](download_images/Image15.jpg)



---

16. [RFK Jr.‘s ‘Make America Healthy Again’ report seems riddled with AI slop, like  “oaicite,” a URL marker that OpenAI applies to responses provided by ChatGPT.](https://www.theverge.com/news/676945/rfk-jr-maha-health-report-ai-slop)  - The Verge![image](download_images/Image16.jpg)



---

17. [OpenAI risks being undercut by cheaper rivals, says star investor Mary Meeker](https://www.ft.com/content/1870c6ba-6d8c-441b-b335-cb29055337f1)  - Financial Times![image](download_images/Image17.jpg)



---

18. [Palantir AI shows up at Fannie Mae to fight mortgage fraud](https://www.theregister.com/2025/05/29/fannie_mae_palantir_ai/)  - The Register

---

19. [Chinese tech groups prepare for AI future without Nvidia](https://www.ft.com/content/bb1315e8-27df-4a93-a4dc-11e2883fdde3)  - Financial Times![image](download_images/Image19.jpg)



---

20. [Hugging Face unveils two new open source humanoid robots](https://techcrunch.com/2025/05/29/hugging-face-unveils-two-new-humanoid-robots/)  - TechCrunch![image](download_images/Image20.jpg)



---

21. [As AI models start exhibiting bad behavior, it's time to start thinking harder about AI safety](https://www.fastcompany.com/91342791/as-ai-models-start-exhibiting-bad-behavior-its-time-to-start-thinking-harder-about-ai-safety)  - Fast Company

---

22. [Not much info on Tesla's small, invitation-only, geofenced robotaxi launch.](https://www.theverge.com/news/676548/tesla-robotaxi-launch-vehicles-date-lack-information)  - The Verge![image](download_images/Image22.jpg)



---

23. [AI could consume more power than Bitcoin by the end of 2025. Seems to be doing more useful work.](https://www.theverge.com/climate-change/676528/ai-data-center-energy-forecast-bitcoin-mining)  - The Verge![image](download_images/Image23.jpg)



---

24. ["Wiles has told associates that her cellphone contacts were hacked... Some of the calls featured a voice that sounded like Wiles, people who heard them said. Government officials think the impersonator used artificial intelligence to imitate Wiles’s voice, some of the people said."](https://www.wsj.com/politics/policy/federal-authorities-probe-effort-to-impersonate-white-house-chief-of-staff-65da0d59?st=SyMfYD&reflink=mobilewebshare_permalink)  - The Wall Street Journal

---

25. [The Darwin Gödel Machine: AI that improves itself by rewriting its own code](https://sakana.ai/dgm/)  - Sakana AI

---

26. [This should be a Netflix miniseries, preferably in Korean](https://news.harvard.edu/gazette/story/2025/05/hey-you-hold-onto-your-humanity-youll-thank-me-later/)  - Harvard Gazette![image](download_images/Image26.jpg)



---

27. 
Reporter: An investigation found the MAHA commission report cites studies that appear not to exist.

Leavitt: I understand there were some formatting issues…It does not negate the substance of the report 

Reporter: Is AI used to put together these reports? 

Leavitt: I can’t speak to that



---

28. [Google’s New A.I.-Powered Search Has Arrived. Proceed With Caution.](https://www.nytimes.com/2025/05/29/technology/personaltech/google-ai-mode-search.html)  - The New York Times

---

29. [GPT-4o displays humanlike commitment and consistency bias, avoids cognitive dissonance, Harvard study finds. The AI is emitting the most probable tokens conditioned on previous responses, so that probably implies some degree of self-consistency.](https://techxplore.com/news/2025-05-gpt-4o-humanlike-cognitive-dissonance.html)  - TechXplore

---

30. 
artificialanalysis.ai/models



---

31. [‘Stop using my voice' - ScotRail's new announcer is my AI clone](https://www.bbc.com/news/articles/cn4q7984nq1o)  - BBC![image](download_images/Image31.jpg)



---

32. [Google is Using AI to Censor Independent Websites](https://travellemming.com/perspectives/ftc-letter-google-censors-indie-publishers-with-ai/)  - Travel Lemming

---

33. [AI personal shoppers hunt down bargain buys](https://techxplore.com/news/2025-05-ai-personal-shoppers-bargain-buys.html)  - TechXplore

---

34. [Want an advanced AI assistant? Concerns about privacy, consent and memory raised by artificial intelligence will become more urgent as more advanced models enter the market.](https://theconversation.com/want-an-advanced-ai-assistant-prepare-for-them-to-be-all-up-in-your-business-253271)  - The Conversation

---

35. [AWS Application Modernization Service Uses AI Agents to Refactor Legacy Code](https://devops.com/aws-application-modernization-service-employs-ai-agents-to-refactor-code/)  - DevOps.com![image](download_images/Image35.jpg)



---

36. [Agentic AI played a decisive role in dismantling DanaBot, a Russian malware platform, accelerating research.](https://venturebeat.com/security/agentic-ai-defeated-danabot-exposing-key-lessons-for-soc-teams/)  - VentureBeat![image](download_images/Image36.jpg)



---

37. [Gemini can now watch Google Drive videos for you](https://www.theverge.com/news/676245/google-drive-gemini-video-summary-feature)  - The Verge![image](download_images/Image37.jpg)



---

38. [Snyk launches AI Trust Platform to address agent vulnerabilities, security in AI-driven software development](https://siliconangle.com/2025/05/28/snyk-launches-ai-trust-platform-address-security-ai-driven-software-development/)  - SiliconANGLE![image](download_images/Image38.jpg)



---

39. [Salesforce Says AI Has Reduced Hiring of Engineers and Customer Service Workers](https://www.bloomberg.com/news/articles/2025-05-28/salesforce-says-ai-has-reduced-hiring-of-engineers-and-customer-service-workers)  - Bloomberg

---

40. [ServiceNow’s “AI employees” already handle 20 % of support work and aim for 80 % coverage of selected roles.](https://greylock.com/greymatter/gen-ai-servicenow/)  - greylock.com

---

41. 
was it trained on 2001: A Space Odyssey? if it doesn't HAL 9000, is it even intelligent ?



---

42. [Given the power to do so, OpenAI o3 model tries to modify its own shutdown script to prevent shutting it down, similar to what Anthropic reported, say researchers](https://www.theregister.com/2025/05/29/openai_model_modifies_shutdown_script/)  - The Register

---

43. 
I feel like you're just showing me what I want to see



---

44. 
Stockholm syndrome or hostage video? TFW you think you're sitting pretty but you're actually the prisoner of the people who are supposed to serve you. He's a little like Zuck gonna be in his bunker after the apocalypse,



---

45. [Nvidia CEO Jensen Huang had a lot to say about the China restrictions and praised President Donald Trump during the company's Q1 earnings call.](https://www.businessinsider.com/nvidia-q1-earnings-call-fy26-takeaways-summary-jensen-huang-china-2025-5?international=true&r=US&IR=T)  - Business Insider![image](download_images/Image45.jpg)



---

46. [Criminals are using AI to create fake documents to trick businesses](https://www.techradar.com/pro/criminals-are-using-ai-to-create-fake-documents-to-trick-businesses)  - TechRadar![image](download_images/Image46.jpg)



---

47. [UK must toughen regulation of facial recognition, say AI experts](https://www.ft.com/content/09ce1755-554e-43c9-90ad-8e07196763b0)  - Financial Times![image](download_images/Image47.jpg)



---

48. [Law firm clients ask why they are not benefiting from AI cost-cutting](https://www.ft.com/content/cba88ff9-41ea-4137-a16e-8e46ec652d30)  - Financial Times![image](download_images/Image48.jpg)



---

49. [The Times and Amazon Announce an A.I. Licensing Deal](https://www.nytimes.com/2025/05/29/business/media/new-york-times-amazon-ai-licensing.html)  - The New York Times

---

50. [Trump’s “One Big Beautiful Bill” could turn the US into an AI Wild West until 2035](https://www.phonearena.com/news/trumps-one-big-beautiful-bill-could-turn-the-us-into-an-ai-wild-west-until-2035_id170833)  - PhoneArena![image](download_images/Image50.jpg)



---

51. [We're cooked](https://www.reddit.com/r/singularity/comments/1ky1r6z/were_cooked_zerocost_ai_demo/)  - Reddit

---

52. [DeepSeek says update offers better reasoning, 'approaches' o3 and Gemini 2.5](https://finance.yahoo.com/news/deepseek-unveils-r1-model-032949885.html)  - Yahoo Finance

---

53. [Why Anthropic’s New AI Model Sometimes Tries to ‘Snitch’](https://www.wired.com/story/anthropic-claude-snitch-emergent-behavior/)  - WIRED![image](download_images/Image53.jpg)



---

54. [The internet thinks this video from Gaza is AI. Here's how we proved it isn't.](https://www.nbcnews.com/tech/tech-news/internet-thinks-video-gaza-ai-s-proved-isnt-rcna209617)  - NBC News![image](download_images/Image54.jpg)



---

55. [J.D. Vance calls AI a 'communist technology.'](https://www.morningstar.com/news/marketwatch/20250528239/jd-vance-calls-ai-a-communist-technology-is-there-partisan-bias-in-new-tech-tools)  - Morningstar

---

56. [Data centre adjustments by Amazon and Microsoft reflect harsh realities: power grids that take years to expand, land speculators inflating prices, and overwhelmed utilities](https://qz.com/ai-data-centers-microsoft-amazon-electric-grid-land-1851782495)  - Quartz![image](download_images/Image56.jpg)



---

57. [Strava turns to AI to catch cheaters. Like anybody cares about the record for some park loop.](https://lifehacker.com/health/strava-is-using-ai-to-detect-cheaters)  - Lifehacker![image](download_images/Image57.jpg)



---

58. [Langchain released a no-code agent building patform](https://github.com/langchain-ai/open-agent-platform/)  - GitHub

---

59. [OpenAI CFO: New Structure Can Pave Way for IPO](https://www.pymnts.com/news/ipo/2025/openai-cfo-new-structure-can-pave-way-ipo/)  - PYMNTS.com![image](download_images/Image59.jpg)



---

60. [a16z says Generative Engine Optimization kills SEO like a disturbed man on the subway](https://a16z.com/geo-over-seo/?utm_source=social&utm_medium=x&utm_campaign=enterprise)  - Andreessen Horowitz![image](download_images/Image60.jpg)



---

61. [How Sam Altman and Jony Ive Will Force A.I. Into Your Life. We'll see, could be a niche early-adopter device, or just throwing spaghetti at the wall to see if it sticks.](https://www.newyorker.com/culture/infinite-scroll/sam-altman-and-jony-ive-will-force-ai-into-your-life)  - The New Yorker![image](download_images/Image61.jpg)



---

62. [search-fu failed me, weights are on Hugging Face](https://huggingface.co/deepseek-ai/DeepSeek-R1-0528/tree/main)  - Hugging Face![image](download_images/Image62.jpg)



---

63. [Alibaba’s healthcare AI model on par with senior physicians in medical exams](https://www.scmp.com/tech/tech-trends/article/3312153/alibabas-healthcare-ai-model-scores-high-senior-level-doctors-medical-exams)  - South China Morning Post![image](download_images/Image63.jpg)



---

64. 
Musk had learned just before Trump’s mid-May Gulf tour that Sam Altman was going to be on the trip and that a deal was in the works, and grew angry about it, according to White House officials. He then said he would also join the trip, and appeared alongside the president in Saudi Arabia.



---

65. [Elon Musk Tried to Block Sam Altman’s Big AI Deal in the Middle East. Musk warned those involved that President Trump wouldn’t bless the OpenAI data-center project in Abu Dhabi unless his xAI company was included in the deal.](https://www.wsj.com/tech/elon-musk-trump-openai-stargate-abu-dhabi-e2689615)  - The Wall Street Journal

---

66. [I understood some of those words](https://www.securityweek.com/the-root-of-ai-hallucinations-physics-theory-digs-into-the-attention-flaw/)  - SecurityWeek

---

67. [E&E News: Is climate change a threat? It depends, says Elon Musk’s AI chatbot.](https://subscriber.politicopro.com/article/eenews/2025/05/27/is-climate-change-a-threat-it-all-depends-says-elon-musks-ai-chatbot-00367904)  - Politico Pro![image](download_images/Image67.jpg)



---

68. [The age of AI layoffs is here.](https://qz.com/ai-layoffs-jobs-microsoft-walmart-tech-workers-1851782194)  - Quartz![image](download_images/Image68.jpg)



---

69. [DeepSeek stealth-upgrades R1 model. No new weights on Hugging Face though, only in app/API apparently.](https://www.techrepublic.com/article/news-deepseek-r1-update-china-ai/)  - TechRepublic

---

70. [AI Is Destroying Gen Z's Chances at a Stable Career](https://futurism.com/ai-gen-z-careers)  - Futurism![image](download_images/Image70.jpg)



---

71. [AI has blind spots, won't replace radiologists anytime soon](https://www.theregister.com/2025/05/28/ai_models_still_not_up/)  - The Register

---

72. [AI could wipe out 50% of entry-level white-collar jobs, Anthropic CEO warns](https://www.businessinsider.com/anthropic-ceo-warning-ai-could-eliminate-jobs-2025-5)  - Business Insider![image](download_images/Image72.jpg)



---

73. [Low-code platform Retool adds AI agent creation tools](https://www.businessinsider.com/ai-application-company-retool-says-ai-here-to-replace-labor-2025-5?international=true&r=US&IR=T)  - Business Insider![image](download_images/Image73.jpg)



---

74. [Trump orders US chip-design software companies like Cadence to stop selling to China](https://www.ft.com/content/def7b2f1-0cda-4742-b1e7-f3316165bd20)  - Financial Times![image](download_images/Image74.jpg)



---

75. [Nvidia reports, stock rises a little](https://www.nytimes.com/2025/05/28/technology/nvidia-earnings-ai-chips.html)  - The New York Times

---

76. [Aussies do be like that](https://www.reddit.com/r/singularity/comments/1kxax4j/this_emotional_support_kangaroo_video_is_going/)  - Reddit

---

77. [di AI been hitting di ganja, pass the spliff and hold on tight](https://www.reddit.com/r/ChatGPT/comments/1kxd5ve/di_breddaz_of_avengjah/)  - Reddit

---

78. [Big Tech Week calendar in NYC next week](https://www.tech-week.com/calendar)  - www.tech-week.com![image](download_images/Image78.jpg)



---

79. [OpenAI wants to be your identity provider. Probably they want to be able to run agents on other SaaS services frictionlessly. Maybe eventually via the Orb.](https://www.cryptopolitan.com/openai-tests-new-sign-in-service/)  - Cryptopolitan

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [20]:
stop_index = 31

In [21]:
output_str = ""
for i in sorted(display_dict.keys()):
    if i >= stop_index:
        break
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))

0. [Urine-based ML test identified prostate cancer biomarkers more accurately than PSA.](https://www.darkdaily.com/2025/05/30/new-prostate-cancer-test-uses-machine-learning-to-efficiently-spot-new-cancer-biomarkers-accurately-and-non-invasively/)  - www.darkdaily.com

---

1. [Carnegie-Mellon study reveals autonomous coding agents succeed on only 30% of tasks, reinforcing need for human collaboration.](https://www.zdnet.com/article/ai-agents-make-great-teammates-but-dont-let-them-code-alone-heres-why/)  - ZDNET![image](download_images/Image1.jpg)



---

2. [Elephant benchmark used Reddit’s AITA to test how much AI models suck up to us](https://www.technologyreview.com/2025/05/30/1117551/this-benchmark-used-reddits-aita-to-test-how-much-ai-models-suck-up-to-us/)  - MIT Technology Review![image](download_images/Image2.jpg)



---

3. [Is media coming for AI with a "Napster moment?"](https://www.hollywoodreporter.com/business/business-news/ai-napster-moment-1236232248/)  - The Hollywood Reporter![image](download_images/Image3.jpg)



---

4. [Huang says AI won't take your job, but someone who uses AI effectively might. Other CEOs chime in. Some categories of jobs have disappeared due to tech: phone operator, typist etc.](https://bgr.com/business/nvidia-ceo-knows-exactly-who-will-take-your-job-and-its-not-just-ai/)  - BGR![image](download_images/Image4.jpg)



---

5. [Could AI make a Scorsese movie? Demis Hassabis and Darren Aronofsky discuss](https://www.ft.com/content/2fff10ea-f87e-4aaa-ae0e-50d17022dfa3)  - Financial Times![image](download_images/Image5.jpg)



---

6. [OpenAI Aims to Offer AI Access Through ‘Ambient Computer Layer,’ Without Screens](https://www.pymnts.com/artificial-intelligence-2/2025/openai-aims-to-offer-ai-access-through-ambient-computer-layer-without-screens/)  - PYMNTS.com![image](download_images/Image6.jpg)



---

7. [People who use AI at work are silently judged more lazy, less competent](https://www.cnet.com/tech/services-and-software/use-ai-at-work-your-coworkers-may-be-judging-you/)  - CNET![image](download_images/Image7.jpg)



---

8. [For Some Recent Graduates, the A.I. Job Apocalypse May Already Be Here](https://www.nytimes.com/2025/05/30/technology/ai-jobs-college-graduates.html)  - The New York Times

---

9. [Firefox will summarize links if you hover and press alt-shift, but it's slow.](https://www.techradar.com/computing/i-tried-firefox-139s-new-ai-generated-link-previews-and-while-i-love-the-idea-im-skipping-them-for-one-key-reason)  - TechRadar![image](download_images/Image9.jpg)



---

10. [Business Insider fires 21% of staff in AI pivot "away from journalism toward greed"](https://www.avclub.com/business-insider-laysoff-20-percent-staff-ai)  - The A.V. Club

---

11. [AI generates navigable 3d environments like game levels.](https://www.zdnet.com/article/this-interactive-ai-video-generator-feels-like-walking-into-a-video-game-how-to-try-it/)  - ZDNET![image](download_images/Image11.jpg)



---

12. [TikTok boosts Polish hard right content ahead of elections](https://globalwitness.org/en/campaigns/digital-threats/tiktok-algorithm-recommends-twice-as-much-hard-right-content-to-users-ahead-of-polish-election/)  - Global Witness![image](download_images/Image12.jpg)



---

13. [CEOs know AI will shrink their teams in some cases 30-40% — they're just too afraid to say it, say 2 software investors](https://finance.yahoo.com/news/ceos-know-ai-shrink-teams-052955673.html)  - Yahoo Finance

---

14. 
AI will replace us all. Also AI:



---

15. [Google fixes bug that led AI Overviews to say it's now 2024](https://techcrunch.com/2025/05/30/google-fixes-bug-that-led-ai-overviews-to-say-its-now-2024/)  - TechCrunch![image](download_images/Image15.jpg)



---

16. [RFK Jr.‘s ‘Make America Healthy Again’ report seems riddled with AI slop, like  “oaicite,” a URL marker that OpenAI applies to responses provided by ChatGPT.](https://www.theverge.com/news/676945/rfk-jr-maha-health-report-ai-slop)  - The Verge![image](download_images/Image16.jpg)



---

17. [OpenAI risks being undercut by cheaper rivals, says star investor Mary Meeker](https://www.ft.com/content/1870c6ba-6d8c-441b-b335-cb29055337f1)  - Financial Times![image](download_images/Image17.jpg)



---

18. [Palantir AI shows up at Fannie Mae to fight mortgage fraud](https://www.theregister.com/2025/05/29/fannie_mae_palantir_ai/)  - The Register

---

19. [Chinese tech groups prepare for AI future without Nvidia](https://www.ft.com/content/bb1315e8-27df-4a93-a4dc-11e2883fdde3)  - Financial Times![image](download_images/Image19.jpg)



---

20. [Hugging Face unveils two new open source humanoid robots](https://techcrunch.com/2025/05/29/hugging-face-unveils-two-new-humanoid-robots/)  - TechCrunch![image](download_images/Image20.jpg)



---

21. [As AI models start exhibiting bad behavior, it's time to start thinking harder about AI safety](https://www.fastcompany.com/91342791/as-ai-models-start-exhibiting-bad-behavior-its-time-to-start-thinking-harder-about-ai-safety)  - Fast Company

---

22. [Not much info on Tesla's small, invitation-only, geofenced robotaxi launch.](https://www.theverge.com/news/676548/tesla-robotaxi-launch-vehicles-date-lack-information)  - The Verge![image](download_images/Image22.jpg)



---

23. [AI could consume more power than Bitcoin by the end of 2025. Seems to be doing more useful work.](https://www.theverge.com/climate-change/676528/ai-data-center-energy-forecast-bitcoin-mining)  - The Verge![image](download_images/Image23.jpg)



---

24. ["Wiles has told associates that her cellphone contacts were hacked... Some of the calls featured a voice that sounded like Wiles, people who heard them said. Government officials think the impersonator used artificial intelligence to imitate Wiles’s voice, some of the people said."](https://www.wsj.com/politics/policy/federal-authorities-probe-effort-to-impersonate-white-house-chief-of-staff-65da0d59?st=SyMfYD&reflink=mobilewebshare_permalink)  - The Wall Street Journal

---

25. [The Darwin Gödel Machine: AI that improves itself by rewriting its own code](https://sakana.ai/dgm/)  - Sakana AI

---

26. [This should be a Netflix miniseries, preferably in Korean](https://news.harvard.edu/gazette/story/2025/05/hey-you-hold-onto-your-humanity-youll-thank-me-later/)  - Harvard Gazette![image](download_images/Image26.jpg)



---

27. 
Reporter: An investigation found the MAHA commission report cites studies that appear not to exist.

Leavitt: I understand there were some formatting issues…It does not negate the substance of the report 

Reporter: Is AI used to put together these reports? 

Leavitt: I can’t speak to that



---

28. [Google’s New A.I.-Powered Search Has Arrived. Proceed With Caution.](https://www.nytimes.com/2025/05/29/technology/personaltech/google-ai-mode-search.html)  - The New York Times

---

29. [GPT-4o displays humanlike commitment and consistency bias, avoids cognitive dissonance, Harvard study finds. The AI is emitting the most probable tokens conditioned on previous responses, so that probably implies some degree of self-consistency.](https://techxplore.com/news/2025-05-gpt-4o-humanlike-cognitive-dissonance.html)  - TechXplore

---

30. 
artificialanalysis.ai/models



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [22]:
import markdown
output_str =""
markdown_extensions = [
    # 'tables',
    # 'fenced_code',
    # 'codehilite',
    'attr_list',
    'def_list',
    # 'footnotes',
    'markdown.extensions.nl2br',
    'markdown.extensions.sane_lists'
]

output_str = ""
output_array = []
for i in sorted(display_dict.keys()):
    md_str=""
    if i >= stop_index:
        break
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    output_array.append(display_dict.get(i))
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

output_df = pd.DataFrame(output_array)
output_df.columns = ['markdown']
output_df = output_df.reset_index()
    
display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
        f.write(html_str)

[Urine-based ML test identified prostate cancer biomarkers more accurately than PSA.](https://www.darkdaily.com/2025/05/30/new-prostate-cancer-test-uses-machine-learning-to-efficiently-spot-new-cancer-biomarkers-accurately-and-non-invasively/)  - www.darkdaily.com

---



![image](download_images/Image1.jpg)

[Carnegie-Mellon study reveals autonomous coding agents succeed on only 30% of tasks, reinforcing need for human collaboration.](https://www.zdnet.com/article/ai-agents-make-great-teammates-but-dont-let-them-code-alone-heres-why/)  - ZDNET

---



![image](download_images/Image2.jpg)

[Elephant benchmark used Reddit’s AITA to test how much AI models suck up to us](https://www.technologyreview.com/2025/05/30/1117551/this-benchmark-used-reddits-aita-to-test-how-much-ai-models-suck-up-to-us/)  - MIT Technology Review

---



![image](download_images/Image3.jpg)

[Is media coming for AI with a "Napster moment?"](https://www.hollywoodreporter.com/business/business-news/ai-napster-moment-1236232248/)  - The Hollywood Reporter

---



![image](download_images/Image4.jpg)

[Huang says AI won't take your job, but someone who uses AI effectively might. Other CEOs chime in. Some categories of jobs have disappeared due to tech: phone operator, typist etc.](https://bgr.com/business/nvidia-ceo-knows-exactly-who-will-take-your-job-and-its-not-just-ai/)  - BGR

---



![image](download_images/Image5.jpg)

[Could AI make a Scorsese movie? Demis Hassabis and Darren Aronofsky discuss](https://www.ft.com/content/2fff10ea-f87e-4aaa-ae0e-50d17022dfa3)  - Financial Times

---



![image](download_images/Image6.jpg)

[OpenAI Aims to Offer AI Access Through ‘Ambient Computer Layer,’ Without Screens](https://www.pymnts.com/artificial-intelligence-2/2025/openai-aims-to-offer-ai-access-through-ambient-computer-layer-without-screens/)  - PYMNTS.com

---



![image](download_images/Image7.jpg)

[People who use AI at work are silently judged more lazy, less competent](https://www.cnet.com/tech/services-and-software/use-ai-at-work-your-coworkers-may-be-judging-you/)  - CNET

---

[For Some Recent Graduates, the A.I. Job Apocalypse May Already Be Here](https://www.nytimes.com/2025/05/30/technology/ai-jobs-college-graduates.html)  - The New York Times

---



![image](download_images/Image9.jpg)

[Firefox will summarize links if you hover and press alt-shift, but it's slow.](https://www.techradar.com/computing/i-tried-firefox-139s-new-ai-generated-link-previews-and-while-i-love-the-idea-im-skipping-them-for-one-key-reason)  - TechRadar

---

[Business Insider fires 21% of staff in AI pivot "away from journalism toward greed"](https://www.avclub.com/business-insider-laysoff-20-percent-staff-ai)  - The A.V. Club

---



![image](download_images/Image11.jpg)

[AI generates navigable 3d environments like game levels.](https://www.zdnet.com/article/this-interactive-ai-video-generator-feels-like-walking-into-a-video-game-how-to-try-it/)  - ZDNET

---



![image](download_images/Image12.jpg)

[TikTok boosts Polish hard right content ahead of elections](https://globalwitness.org/en/campaigns/digital-threats/tiktok-algorithm-recommends-twice-as-much-hard-right-content-to-users-ahead-of-polish-election/)  - Global Witness

---

[CEOs know AI will shrink their teams in some cases 30-40% — they're just too afraid to say it, say 2 software investors](https://finance.yahoo.com/news/ceos-know-ai-shrink-teams-052955673.html)  - Yahoo Finance

---


AI will replace us all. Also AI:



---



![image](download_images/Image15.jpg)

[Google fixes bug that led AI Overviews to say it's now 2024](https://techcrunch.com/2025/05/30/google-fixes-bug-that-led-ai-overviews-to-say-its-now-2024/)  - TechCrunch

---



![image](download_images/Image16.jpg)

[RFK Jr.‘s ‘Make America Healthy Again’ report seems riddled with AI slop, like  “oaicite,” a URL marker that OpenAI applies to responses provided by ChatGPT.](https://www.theverge.com/news/676945/rfk-jr-maha-health-report-ai-slop)  - The Verge

---



![image](download_images/Image17.jpg)

[OpenAI risks being undercut by cheaper rivals, says star investor Mary Meeker](https://www.ft.com/content/1870c6ba-6d8c-441b-b335-cb29055337f1)  - Financial Times

---

[Palantir AI shows up at Fannie Mae to fight mortgage fraud](https://www.theregister.com/2025/05/29/fannie_mae_palantir_ai/)  - The Register

---



![image](download_images/Image19.jpg)

[Chinese tech groups prepare for AI future without Nvidia](https://www.ft.com/content/bb1315e8-27df-4a93-a4dc-11e2883fdde3)  - Financial Times

---



![image](download_images/Image20.jpg)

[Hugging Face unveils two new open source humanoid robots](https://techcrunch.com/2025/05/29/hugging-face-unveils-two-new-humanoid-robots/)  - TechCrunch

---

[As AI models start exhibiting bad behavior, it's time to start thinking harder about AI safety](https://www.fastcompany.com/91342791/as-ai-models-start-exhibiting-bad-behavior-its-time-to-start-thinking-harder-about-ai-safety)  - Fast Company

---



![image](download_images/Image22.jpg)

[Not much info on Tesla's small, invitation-only, geofenced robotaxi launch.](https://www.theverge.com/news/676548/tesla-robotaxi-launch-vehicles-date-lack-information)  - The Verge

---



![image](download_images/Image23.jpg)

[AI could consume more power than Bitcoin by the end of 2025. Seems to be doing more useful work.](https://www.theverge.com/climate-change/676528/ai-data-center-energy-forecast-bitcoin-mining)  - The Verge

---

["Wiles has told associates that her cellphone contacts were hacked... Some of the calls featured a voice that sounded like Wiles, people who heard them said. Government officials think the impersonator used artificial intelligence to imitate Wiles’s voice, some of the people said."](https://www.wsj.com/politics/policy/federal-authorities-probe-effort-to-impersonate-white-house-chief-of-staff-65da0d59?st=SyMfYD&reflink=mobilewebshare_permalink)  - The Wall Street Journal

---

[The Darwin Gödel Machine: AI that improves itself by rewriting its own code](https://sakana.ai/dgm/)  - Sakana AI

---



![image](download_images/Image26.jpg)

[This should be a Netflix miniseries, preferably in Korean](https://news.harvard.edu/gazette/story/2025/05/hey-you-hold-onto-your-humanity-youll-thank-me-later/)  - Harvard Gazette

---


Reporter: An investigation found the MAHA commission report cites studies that appear not to exist.

Leavitt: I understand there were some formatting issues…It does not negate the substance of the report 

Reporter: Is AI used to put together these reports? 

Leavitt: I can’t speak to that



---

[Google’s New A.I.-Powered Search Has Arrived. Proceed With Caution.](https://www.nytimes.com/2025/05/29/technology/personaltech/google-ai-mode-search.html)  - The New York Times

---

[GPT-4o displays humanlike commitment and consistency bias, avoids cognitive dissonance, Harvard study finds. The AI is emitting the most probable tokens conditioned on previous responses, so that probably implies some degree of self-consistency.](https://techxplore.com/news/2025-05-gpt-4o-humanlike-cognitive-dissonance.html)  - TechXplore

---


artificialanalysis.ai/models



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

8260

In [23]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import RootModel
import os

from pydantic import BaseModel, RootModel, Field
from typing import List

class ItemList(BaseModel):
    """List of story indexes"""
    items: List[int] = Field(description="The ordered list of story indexes")


system_prompt = """
# ROLE
You are a newsletter editor.

# TASK
Group and reorder the items in the Markdown newsletter supplied below so that:
1. Items that cover similar companies, individuals, technologies or topics are **next to each other** (i.e., logically grouped).
2. Groups appear from **most important** to **least important** according to the **IMPORTANCE FACTORS** listed below.
3. Importance of groups is determined by the most imporant story 

# HOW TO GROUP
- If two or more items discuss the same organization, person, technology, or topic, keep them all together.
- Within any group, order by importance using **IMPORTANCE FACTORS**.

# HOW TO ORDER
- Lead with stories that have highest importance per the **IMPORTANCE FACTORS** below
- Follow with narrower‑scope or opinion pieces.
- End with lighter or human‑interest items.

# IMPORTANCE FACTORS
1 Magnitude of impact : large user base, $ at stake, broad social reach
2 Novelty : breaks conceptual ground, not a minor iteration
3 Authority : reputable institution, peer-review, regulatory filing, on-record executive
4 Verifiability : code, data, benchmarks, or other concrete evidence provided
5 Timeliness : early signal of an important trend or shift
6 Breadth : cross-industry / cross-disciplinary / international implications
7 Strategic consequence : shifts competitive, power, or policy dynamics
8 Financial materiality : clear valuation or growth or revenue impact
9 Risk & safety : raises or mitigates critical alignment, security, or ethical risk
10 Actionability : informs concrete decisions for investors, policymakers, practitioners
11 Longevity : likely to matter or be referred to in coming days, weeks, or months
12 Independent corroboration : confirmed by multiple sources or datasets
13 Clarity : enough technical/context detail to judge merit; minimal hype

# INPUT FORMAT
You will receive a JSON array of items, with an index and markdown summary of the item.

# OUTPUT FORMAT
- Return **only** the reordered indexes using the provided schema.
- Do **not** alter the wording of individual items (other than correcting spelling and punctuation, and normalizing spaces or line breaks as needed).
- Insert one blank line between each logical group for readability.
- Do not output anything other than the newsletter itself—no headers, explanations, or commentary.

"""

user_prompt = """
{input_text}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

model = ChatOpenAI(
        model="gpt-4.1",  
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
# Create the chain
chain = prompt_template | model.with_structured_output(ItemList)
response = chain.invoke({"input_text": output_df.to_json(orient="records")})


In [24]:
len(response.items)


31

In [25]:
output_str = ""
for i in response.items:
    md_str = ""
#     output_str += f"{i}\n\n"
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
    f.write(html_str)
    



![image](download_images/Image19.jpg)

[Chinese tech groups prepare for AI future without Nvidia](https://www.ft.com/content/bb1315e8-27df-4a93-a4dc-11e2883fdde3)  - Financial Times

---



![image](download_images/Image4.jpg)

[Huang says AI won't take your job, but someone who uses AI effectively might. Other CEOs chime in. Some categories of jobs have disappeared due to tech: phone operator, typist etc.](https://bgr.com/business/nvidia-ceo-knows-exactly-who-will-take-your-job-and-its-not-just-ai/)  - BGR

---

[CEOs know AI will shrink their teams in some cases 30-40% — they're just too afraid to say it, say 2 software investors](https://finance.yahoo.com/news/ceos-know-ai-shrink-teams-052955673.html)  - Yahoo Finance

---

[For Some Recent Graduates, the A.I. Job Apocalypse May Already Be Here](https://www.nytimes.com/2025/05/30/technology/ai-jobs-college-graduates.html)  - The New York Times

---



![image](download_images/Image7.jpg)

[People who use AI at work are silently judged more lazy, less competent](https://www.cnet.com/tech/services-and-software/use-ai-at-work-your-coworkers-may-be-judging-you/)  - CNET

---



![image](download_images/Image6.jpg)

[OpenAI Aims to Offer AI Access Through ‘Ambient Computer Layer,’ Without Screens](https://www.pymnts.com/artificial-intelligence-2/2025/openai-aims-to-offer-ai-access-through-ambient-computer-layer-without-screens/)  - PYMNTS.com

---



![image](download_images/Image17.jpg)

[OpenAI risks being undercut by cheaper rivals, says star investor Mary Meeker](https://www.ft.com/content/1870c6ba-6d8c-441b-b335-cb29055337f1)  - Financial Times

---



![image](download_images/Image5.jpg)

[Could AI make a Scorsese movie? Demis Hassabis and Darren Aronofsky discuss](https://www.ft.com/content/2fff10ea-f87e-4aaa-ae0e-50d17022dfa3)  - Financial Times

---



![image](download_images/Image11.jpg)

[AI generates navigable 3d environments like game levels.](https://www.zdnet.com/article/this-interactive-ai-video-generator-feels-like-walking-into-a-video-game-how-to-try-it/)  - ZDNET

---



![image](download_images/Image20.jpg)

[Hugging Face unveils two new open source humanoid robots](https://techcrunch.com/2025/05/29/hugging-face-unveils-two-new-humanoid-robots/)  - TechCrunch

---



![image](download_images/Image2.jpg)

[Elephant benchmark used Reddit’s AITA to test how much AI models suck up to us](https://www.technologyreview.com/2025/05/30/1117551/this-benchmark-used-reddits-aita-to-test-how-much-ai-models-suck-up-to-us/)  - MIT Technology Review

---

[GPT-4o displays humanlike commitment and consistency bias, avoids cognitive dissonance, Harvard study finds. The AI is emitting the most probable tokens conditioned on previous responses, so that probably implies some degree of self-consistency.](https://techxplore.com/news/2025-05-gpt-4o-humanlike-cognitive-dissonance.html)  - TechXplore

---



![image](download_images/Image1.jpg)

[Carnegie-Mellon study reveals autonomous coding agents succeed on only 30% of tasks, reinforcing need for human collaboration.](https://www.zdnet.com/article/ai-agents-make-great-teammates-but-dont-let-them-code-alone-heres-why/)  - ZDNET

---

[As AI models start exhibiting bad behavior, it's time to start thinking harder about AI safety](https://www.fastcompany.com/91342791/as-ai-models-start-exhibiting-bad-behavior-its-time-to-start-thinking-harder-about-ai-safety)  - Fast Company

---



![image](download_images/Image23.jpg)

[AI could consume more power than Bitcoin by the end of 2025. Seems to be doing more useful work.](https://www.theverge.com/climate-change/676528/ai-data-center-energy-forecast-bitcoin-mining)  - The Verge

---

[Google’s New A.I.-Powered Search Has Arrived. Proceed With Caution.](https://www.nytimes.com/2025/05/29/technology/personaltech/google-ai-mode-search.html)  - The New York Times

---



![image](download_images/Image15.jpg)

[Google fixes bug that led AI Overviews to say it's now 2024](https://techcrunch.com/2025/05/30/google-fixes-bug-that-led-ai-overviews-to-say-its-now-2024/)  - TechCrunch

---



![image](download_images/Image9.jpg)

[Firefox will summarize links if you hover and press alt-shift, but it's slow.](https://www.techradar.com/computing/i-tried-firefox-139s-new-ai-generated-link-previews-and-while-i-love-the-idea-im-skipping-them-for-one-key-reason)  - TechRadar

---

[Business Insider fires 21% of staff in AI pivot "away from journalism toward greed"](https://www.avclub.com/business-insider-laysoff-20-percent-staff-ai)  - The A.V. Club

---



![image](download_images/Image3.jpg)

[Is media coming for AI with a "Napster moment?"](https://www.hollywoodreporter.com/business/business-news/ai-napster-moment-1236232248/)  - The Hollywood Reporter

---



![image](download_images/Image12.jpg)

[TikTok boosts Polish hard right content ahead of elections](https://globalwitness.org/en/campaigns/digital-threats/tiktok-algorithm-recommends-twice-as-much-hard-right-content-to-users-ahead-of-polish-election/)  - Global Witness

---

["Wiles has told associates that her cellphone contacts were hacked... Some of the calls featured a voice that sounded like Wiles, people who heard them said. Government officials think the impersonator used artificial intelligence to imitate Wiles’s voice, some of the people said."](https://www.wsj.com/politics/policy/federal-authorities-probe-effort-to-impersonate-white-house-chief-of-staff-65da0d59?st=SyMfYD&reflink=mobilewebshare_permalink)  - The Wall Street Journal

---



![image](download_images/Image16.jpg)

[RFK Jr.‘s ‘Make America Healthy Again’ report seems riddled with AI slop, like  “oaicite,” a URL marker that OpenAI applies to responses provided by ChatGPT.](https://www.theverge.com/news/676945/rfk-jr-maha-health-report-ai-slop)  - The Verge

---


Reporter: An investigation found the MAHA commission report cites studies that appear not to exist.

Leavitt: I understand there were some formatting issues…It does not negate the substance of the report 

Reporter: Is AI used to put together these reports? 

Leavitt: I can’t speak to that



---

[Urine-based ML test identified prostate cancer biomarkers more accurately than PSA.](https://www.darkdaily.com/2025/05/30/new-prostate-cancer-test-uses-machine-learning-to-efficiently-spot-new-cancer-biomarkers-accurately-and-non-invasively/)  - www.darkdaily.com

---

[Palantir AI shows up at Fannie Mae to fight mortgage fraud](https://www.theregister.com/2025/05/29/fannie_mae_palantir_ai/)  - The Register

---

[The Darwin Gödel Machine: AI that improves itself by rewriting its own code](https://sakana.ai/dgm/)  - Sakana AI

---



![image](download_images/Image22.jpg)

[Not much info on Tesla's small, invitation-only, geofenced robotaxi launch.](https://www.theverge.com/news/676548/tesla-robotaxi-launch-vehicles-date-lack-information)  - The Verge

---



![image](download_images/Image26.jpg)

[This should be a Netflix miniseries, preferably in Korean](https://news.harvard.edu/gazette/story/2025/05/hey-you-hold-onto-your-humanity-youll-thank-me-later/)  - Harvard Gazette

---


AI will replace us all. Also AI:



---


artificialanalysis.ai/models



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

8260

In [21]:
pyperclip.copy( output_str )


In [ ]:
date_str = data.feed[0].post.record.created_at
dt = datetime.fromisoformat(date_str.rstrip('Z'))
dt

In [ ]:

# Get the current datetime
now = datetime.now()

# Create a timedelta of 30 hours
delta = timedelta(hours=30)

# Subtract 30 hours from the current datetime
thirty_hours_ago = now - delta

print(thirty_hours_ago)


In [ ]:
from typing import TypedDict, Annotated

class Point2D(TypedDict, total=False):
    x: int
    y: int
    label: str

a: Point2D = {'x': 1, 'y': 2, }  # 


In [ ]:
a

In [ ]:
import numpy as np

def decay_exponential(t, N0):
    k = np.log(2)  
    return N0 * np.exp(-k * t)

2*decay_exponential(4,1)-1

In [ ]:
```mermaid
flowchart LR

A[Hard] -->|Text| B(Round)
B --> C{Decision}
C -->|One| D[Result 1]
C -->|Two| E[Result 2]


In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

dt_str = '2025-05-24 16:00'
naive_dt = datetime.strptime(dt_str, '%Y-%m-%d %H:%M')

# CORRECT: Attach US Eastern timezone *with DST handled*
eastern_dt = naive_dt.replace(tzinfo=ZoneInfo("America/New_York"))

# Convert to UTC
utc_dt = eastern_dt.astimezone(ZoneInfo("UTC"))
print(utc_dt)

In [ ]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions

import chromadb

# Set up the client
client = chromadb.PersistentClient(
    path="chromadb/chroma_articles/", settings=Settings(allow_reset=True)
)


# Get the collection
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    model_name="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY")
)

collection = client.get_collection("articles", embedding_function=openai_ef)

# Count the documents
num_docs = collection.count()
print(f"Number of documents in 'articles': {num_docs}")

In [ ]:
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

before_date = '2025-05-25 16:30'
if before_date:
    naive_dt = datetime.strptime(before_date, '%Y-%m-%d %H:%M')
    eastern_dt = naive_dt.replace(
        tzinfo=ZoneInfo("America/New_York"))
    utc_dt = eastern_dt.astimezone(ZoneInfo("UTC"))
    before_date = utc_dt.strftime('%Y-%m-%d %H:%M')


In [ ]:
doc = collection.get(limit=1)
doc_str = doc['documents'][0]
doc_str2 = "\n".join(doc_str.split("\n")[12:])

In [ ]:
results = collection.query(
    query_texts=[doc_str2],
    n_results=1,  # number of similar documents to return
    include=["distances", "metadatas"],
#     where={'created': {'$lt': 2025-05-25 20:30'}
)

In [ ]:
results


In [ ]:
before_date


In [ ]:
                where_filter = {"created": f"$lt: {before_date}"}


In [ ]:
where_filter

In [ ]:
from typing import List 
import numpy as np 

def mmr(
    ratings: List[float],
    doc_embeddings: List[np.ndarray],
    lambda_param: float = 0.5,
    top_k: int = 50
) -> List[int]:
    """MMR (Maximal Marginal Relevance) algorithm for selecting a diverse set of documents.
    Combines normalized score and novelty vs previous items

    Args:
        doc_embeddings (List[np.ndarray]): embeddings of documents
        ratings (List[float]): ratings of documents, should be positive, we will scale to max 1
        lambda_param (float, optional): balance between rating and novelty. Defaults to 0.5.
        top_k (int, optional): number of documents to select. Defaults to 50.

    Returns:
        List[int]: indices of selected documents
    """
    selected = []
    candidates = list(range(len(doc_embeddings)))

    ratings = [rating / max(ratings) for rating in ratings]

    while len(selected) < top_k and candidates:
        if not selected:  # none yet
            # start with highest scoring document
            idx = np.argmax(ratings)
            selected.append(idx)  # add to selected
            candidates.remove(idx)  # remove from candidates
            continue

        scores = []
        for idx in candidates:
            rating = ratings[idx]
            # find most similar selected document to this doc idx
            # first array has 1 element so we get a 1x array
            max_sim_to_selected = max(cosine_similarity(
                [doc_embeddings[idx]],  # array with one doc
                [doc_embeddings[i]
                    for i in selected]  # array with all selected docs
            )[0])
            mmr_score = lambda_param * rating - \
                (1 - lambda_param) * max_sim_to_selected
            scores.append((mmr_score, idx))

        # Select candidate with highest MMR score
        # scores is a list of (score, idx), max does lexicographic sort so we get the highest score
        _, best_idx = max(scores)
        selected.append(best_idx)
        candidates.remove(best_idx)

    return selected